In [2]:
import os
import pandas as pd
import faiss
import pickle
from sentence_transformers import SentenceTransformer
import openai
import numpy as np


In [3]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   분류번호    6022 non-null   object
 1   사무명     6022 non-null   object
 2   사무개요    6022 non-null   object
 3   신청유형    6022 non-null   object
 4   신청자 자격  6020 non-null   object
 5   수수료     6021 non-null   object
 6   수수료내역   6018 non-null   object
 7   소관부처    6022 non-null   object
 8   담당부서    6022 non-null   object
 9   접수방법    6022 non-null   object
 10  접수/처리   6013 non-null   object
 11  근거법령    5981 non-null   object
 12  신청서     5124 non-null   object
 13  발급물 내용  918 non-null    object
 14  구비서류    5372 non-null   object
dtypes: object(15)
memory usage: 705.8+ KB


In [5]:
pd.set_option('display.max_colwidth', 1000)

In [11]:
df.head(50)

,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN
1,9740000-0009,공탁신청(공탁물지급청구 포함),"공탁 신청, 공탁금의 지급을 위한 민원입니다. 공탁은 공탁자가 법령에 규정된 원인에 따라 금전, 유가증권, 그 밖의 물품을 공탁소에 맡기고, 일정한 자가 수령하도록 함으로써 채무의 변제, 손해의 담보, 집행목적물의 관리 및 집행당사자에의 교부, 목적물의 보관 등의 목적을 달성하게 하는 제도입니다. ※ 공탁의 종류에 따라 공탁신청시 또는 지급청구시 신청서와 첨부서류가 다르므로, 구체적인 첨부서류 등은 해당 공탁소로 문의하여 주시기 바랍니다.",승인,누구나신청가능,수수료 없음,수수료 없음,대법원,사법등기국,"방문, 인터넷",1. 유형없음(총 0일)\n*접수:지방법원.지원(0일)\n*경유:\n*협의:\n*처리:지방법원.지원(0일),"1. 공탁법(제4조, 제9조) \n2. 공탁규칙(제20조, 제21조, 제23조, 제24조, 제32조, 제33조, 제34조, 제37조, 제38조) \n3. 공탁사무 문서양식에 관한 예규(제1조, 제2조)",금전공탁서(변제 등) 공탁사무 문서양식에 관한 예규 : 별지서식 1호의 1\n,NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 인감증명서 신분증 가압류,압류결정문 사본 담보제공명령서 사본\n< 공무원확인사항 >\n(2) 외국인등록사실증명\n(3) 국내거소신고사실증명\n(4) 사업자등록증명\n(5) 주민등록등(초)본"
2,9740000-0008,친양자입양취소신고,가정법원의 허가를 얻어 친양자 입양을 취소하기 위해 신청하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 즉시)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:시.구.읍.면(즉시),"1. 가족관계의 등록 등에 관한 법률(제69조, 제70조)","제9호, 친양자 입양취소신고서\n",NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자입양취소당사자의 가족관계등록부의 기본증명서, 친양자입양관계증명서 각1통(전산정보처리에 의해 확인할 수 없는 경우에 한함)\n(2) 친양자 입양취소재판의 등본 및 확정증명 각1부\n(3) 신고인의 신분증명서 사본 첨부(우편신고의 경우에 한함)\n(4) 신고인이 출석한 경우의 신고인 신분증명서, 제출인이 출석한 경우 제출인의 신분증명서\n< 공무원확인사항 >\n(5) 가족관계등록부"
3,9740000-0007,친양자파양신고,가정법원의 허가를 얻어 친양자입양을 파양하고자 하는 사람이 재판의 확정일부터 1개월 이내에 가족관계등록 관서의 장에게 신고하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:시.구.읍.면(지체없이),1. 가족관계의 등록 등에 관한 법률(제69조),"제7호, 친양자파양신고서\n",NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자파양당사자의 가족관계등록부의 기본증명서, 친양자입양관계증명서 각1통(전산정보처리조지에 의하여 그 내용을 확인할 수 없는 경우에 한함)\n(2) 친양자 파양재판의 등본 및 확정증명 각 1부\n(3) 신고인의 신분증명서 사본 첨부(우편신고의 경우에 한함)\n(4) 신고인이 출석한 경우에는 신고인의 신분증명서, 제출인이 출석한 경우에는 제출인의 신분증명서\n< 공무원확인사항 >\n(5) 가족관계등록부"
4,9740000-0006,성·본계속사용신고,부가 혼인외 자를 인지할 때 인지 전의 자녀의 성과 본을 유지할 경우 그 취지를 가족관계등록 관서의 장에게 신고하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음(총 0일)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:시.구.읍.면(0일),1. 가족관계의 등록 등에 관한 법률(제55조 제1항),"제3호, 성·본계속사용신고서\n",NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 성·본계속사용자의 가족관계등록부의 가족관계증명서 1통. (전산정보처리조직에 의하여 그 내용을 확인할 수 없을 경우에 한함)\n(2) 성.본계속사용 법원 재판서 등본 1부\n(3) 성.본계속사용 협의사실을 증명하는 서류 1부 및 신고인의 신분증 지참(부모가 협의한 경 우로서 부모 중 일방이 신고할 경우에 한함)\n(4) 신고인의 신분증명서 사본 1부(우편으로 신고하는 경우에 한함) ※신고인이 출석한 경우에는 신고인의 신분증명서, 제출인이 출석한 경우에는 제출인의 신분증명서\n(5) 부모의 협의에 의한 경우의 추가 구비서류 ( 붙임의 신고서 서식 참조)\n< 공무원확인사항 >\n(6) 가족관계등록부"
5,9740000-0005,성·본변경신고,자의 복리를 위하여 가정법원의 허가를 얻어 자녀의 성.본을 변경하고자 하는 자가 재판확정일부터 1개월 이내에 가족관계등록 관서의 장에게 신고하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:시.구.읍.면(지체없이),1. 가족관계의 등록 등에 관한 법률(제100조 제1항),"제34호, 성·본변경신고서\n",NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 성본변경신고서\n(2) 가정법원의 허가재판 등본 1부\n(3) 신고인의 신분증명서 사본 첨부(우편신고의 경우에 한함)\n(4) 신고인이 출석한 경우에는 신고인의 신분증명서, 제출인이 출석한 경우에는 제출인의 신분증명서"
6,9740000-0004,가족관계등록부등의 증명서 발급,"국민 개개인의 출생, 입양, 혼인, 사망 등 가족관계의 발생 및 변동에 관한 증명서를 발급받기 위한 민원입니다. 가족관계증명서, 기본증명서, 혼인관계증명서, 입양관계증명서, 친양자입양관계증명서가 있으며 각각의 증명서에 대하여 일반·상세·특정증명서로 구분하여 발급받을 수 있습니다. 원칙적으로 현재의 신분관계 등 필수적인 정보만이 기재된 일반증명서 또는 신청인이 선택한 정보만이 기재된 특정증명서를 사용하고, 과거의 신분관계 등 전체의 정보가 기재된 상세증명서는 특별한 이유가 있는 경우에만 사용합니다. 일반증명서의 기재사항은 다음과 같습니다. 본인의 등록기준지, 성명, 성별, 본, 출생연월일, 주민등록번호는 공통사항입니다. 1. 가족관계증명서: 본인과 부모, 배우자, 생존한 현재의 혼인 중의 자녀에 관한 사항 2. 기본증명서: 본인의 출생, 사망, 국적상실에 관한 사항 3. 혼인관계증명서: 본인 및 배우자에 관한 사항과 현재의 혼인에 관한 사항 4. 입양관계증명서: 본인 및 친생부모·양부모 또는 양자에 관한 사항과 현재의 입양에 관한 사항 5. 친양자입양관계증명서: 본인 및 친생부모·양부모 또는 친양자에 관한 사항과 현재의 친양자입양에 관한 사항 ※ 대법원 전자가족관계등록시스템(efamily.scourt.go.kr)에서 발급받을 수 있습니다.",증명,본인또는대리인신청,사례별 수수료,"증명서 1통당 1,000원 / 인터넷으로 발급받을 때는 무료임",대법원,가족관계등록과,"방문, 인터넷, 무인발급기","1. 유형없음( 즉시)\n*접수:동, 시.구.읍.면(0일)\n*경유:\n*협의:\n*처리:동, 시.구.읍.면(즉시)",1. 가족관계의 등록 등에 관한 법률(제14조),"별지 제11호 서식, 가족관계 등록사항별 증명서 교부 등 신청서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 신분증 지참
7,9740000-0003,건물등기사항증명서 발급,건물에 관한 등기기록 사항의 전부 또는 일부를 증명하는 서면을 발급받기 위한 민원입니다. 건물등기사항증명서를 발급받으면 건물에 관한 부동산의 표시사항과 권리관계를 자세히 

In [10]:
# "지문등 사전등록 신청"이 포함된 행만 필터링
result = df[df['사무명'].str.contains("지문등 사전등록 신청", na=False)]

# 결과 출력
result.head()

,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
4581,1320000-0052,지문등 사전등록 신청,실종아동등의 예방 및 조속한 발견과 복귀를 위하여 아동등의 보호자가 신청하는 경우 아동등의 지문 및 얼굴 등에 관한 정보를 미리 등록하는 민원사무입니다,등록,법령상의자격이있는 자(제3자),수수료 없음,수수료 없음,경찰청,청소년보호과,"방문, 우편, 전화, FAX, 인터넷, 모바일, 현장제출 및 기타","1. 유형없음(총 0일)\n*접수:경찰서, 파출소(0일)\n*경유:\n*협의:\n*처리:경찰서(0일)",1. 실종아동등의 보호 및 지원에 관한 법률(제7조의2 제1항) \n2. 실종아동등의 발견 및 유전자검사 등에 관한 규칙(제3조 제2항),아동등 사전등록신청서 실종아동등의 발견 및 유전자검사 등에 관한 규칙 : 별지서식 1호\n,NaN,"1. 유형없음\n< 공무원확인사항 >\n(1) 주민등록표 등본\n(2) 장애인증명서 (등록대상 아동등이 지적장애인, 자폐성장애인 또는 정신장애인인 경우만 해당합니다)"


In [12]:
# "지문등 사전등록 신청"이 포함된 행만 필터링
result = df[df['사무명'].str.contains("등본", na=False)]

# 결과 출력
result.head()

,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN
334,1613000-0006,"공유토지분할에 관한 서류의 열람, 등본발급","공유토지(건물이 있는경우에 한함) 분할을 간편하게 가능하도록하여 토지소유권행사의 편익을 도모하기 위한 공유토지분할에 관한 서류의 열람, 등본발급 신청을 하는민원사무입니다.",교부,누구나신청가능,수수료 없음,수수료 없음,국토교통부,공간정보제도과,방문,1. 유형없음(총 0일)\n*접수:시군구 및 읍면동 출장소(0일)\n*경유:시군구 및 읍면동 출장소(0일)\n*협의:\n*처리:시군구 및 읍면동 출장소(0일),1. 공유토지분할에 관한 특례법 시행령(제12조의2),"공유토지분할에 관한 서류의 열람, 등본발급 신청서 공유토지분할에 관한 특례법 시행령(폐지) : 별지서식 5호\n",NaN,NaN
1015,1520000-0349,어업권원부 열람 및 등본(초본) 발급,어업권원부열람 및 등본(초본) 교부를 받고자 하는 자가 신청서에 의거한 내용 심사 받음으로 열람 및 발급받는 민원사무입니다.,교부,누구나신청가능,조례별 수수료,조례로 정함,해양수산부,어업정책과,"방문, 우편, 전화, FAX, 민원우편, 인터넷",1. 유형없음(총 0일)\n*접수:시.군.구(0일)\n*경유:\n*협의:\n*처리:시.군.구(0일),1. 수산업법(제16조) \n2. 어업ㆍ양식업등록령(제17조제1항),NaN,NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 신청서
1101,1520000-0186,어선원부열람 및 등본발급,어선법시행규칙에 의한 어선원부열람 및 등본교부를 신청하는 민원사무입니다.,교부,누구나신청가능,단일 수수료,1건당 800원,해양수산부,어선안전정책과,"방문, 우편, 전화, FAX, 민원우편, 인터넷, 무인발급기",1. 유형없음(총 0일)\n*접수:시.군.구(0일)\n*경유:\n*협의:\n*처리:시.군.구(0일),1. 어선법 시행규칙(제30조),NaN,NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 구비서류 없음
1148,1520000-0107,선박원부 등본(초본) 발급(열람),선박등록사항을 기재한 선박원부의 등본·초본을 교부 받거나 선박원부의 열람을 하고자 하는 자가 지방해양수산청장에게 신청하는 민원사무입니다.,교부,누구나신청가능,사례별 수수료,"방문, 우편 발급시 100원, 전자민원(인터넷) 발급시 무료",해양수산부,해사안전정책과,"방문, 우편, 민원우편, 인터넷",1. 유형없음(총 0일)\n*접수:지방해양수산청(0일)\n*경유:\n*협의:\n*처리:지방해양수산청(0일),1. 선박법 시행규칙(제13조),NaN,NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 구비서류 없음


In [14]:
# "지문등 사전등록 신청"이 포함된 행만 필터링
result = df[df['사무개요'].str.contains("도로", na=False)]

# 결과 출력
result.head()

,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
341,1611000-0056,건축물도로명주소 변경ㆍ정정,건축물의 도로명주소가 변경되거나 잘못 기재된 사항을 바로 잡고자 하는때 신청하는 민원사무입니다.,등록,본인또는대리인신청,수수료 없음,수수료 없음,국토교통부,건축정책과,"방문, 우편, 인터넷","1. 유형없음(총 0일)\n*접수:시.군.구, 제주특별자치도, 특별자치시(0일)\n*경유:\n*협의:\n*처리:시.군.구, 제주특별자치도, 특별자치시(0일)","1. 건축법(제38조) \n2. 건축법 시행령(제25조) \n3. 건축물대장의 기재 및 관리 등에 관한 규칙(제20조의2 제1항, 제21조 제3항 별지 제17호의2)","건축물도로명주소 (변경, 정정) 신청서 건축물대장의 기재 및 관리 등에 관한 규칙 : 별지서식 17호의 2\n",NaN,1. 변경 신청의 경우\n< 민원인 제출서류 >\n(1) 건축물 소유자의 동의서(변경되는 건축물의 소유자와 거주자가 다르면서 거주자가 신청하는 경우에 한정한다.)\n< 공무원확인사항 >\n(2) 도로명주소개별대장\n2. 정정 신청의 경우\n< 민원인 제출서류 >\n(1) 없음\n< 공무원확인사항 >\n(2) 도로명주소 개별대장
350,1611000-0045,도로점용허가 변경,도로점용허가를 받은 내용이 변경되는 경우 신청하는 민원사무입니다.,허가,본인또는대리인신청,기타,"1,000원 또는 행정청이 속하는 지방자치단체의 조례로 정하는 금액",국토교통부,도로관리과,"방문, 우편, FAX, 민원우편, 인터넷","1. 지방국토관리청, 국토관리사무소 소관(일반점용,일시점용,공작물설치,도로굴착)(총 7일)\n*접수:국토관리사무소, 지방국토관리청(0일)\n*경유:\n*협의:\n*처리:지방국토관리청, 국토관리사무소(7일)\n2. 지방국토관리청, 국토관리사무소 소관(아치 및 육교 사용)(총 7일)\n*접수:지방국토관리청, 국토관리사무소(0일)\n*경유:\n*협의:\n*처리:지방국토관리청, 국토관리사무소(7일)\n3. 특별시·특별자치시·특별자치도, 광역시·도 소관(일반점용)(총 5일)\n*접수:특별자치도, 서울특별시(동), 특별자치시, 광역시·도(구·읍·면)(0일)\n*경유:\n*협의:\n*처리:특별자치도, 광역시·도(시·군), 서울특별시(구), 특별자치시(5일)\n4. 특별시·특별자치시·특별자치도, 광역시·도 소관(일시점용)(총 2일)\n*접수:특별자치도, 광역시·도(구·시·읍·면), 서울특별시(동), 특별자치시(0일)\n*경유:\n*협의:\n*처리:서울특별시, 광역시·도(구·시·읍·면), 특별자치도, 특별자치시(2일)\n5. 특별시·특별자치시·특별자치도, 광역시·도소관(아치 및 육교 사용)(총 4일)\n*접수:특별자치도, 광역시·도(시·군), 서울특별시, 특별자치시(0일)\n*경유:\n*협의:\n*처리:특별자치도, 특별자치시, 광역시·도(시·군), 서울특별시(4일)\n6. 특별시 소관(공작물의 설치)(총 10일)\n*접수:서울특별시(구)(0일)\n*경유:\n*협의:\n*처리:서울특별시(구)(10일)\n7. 광역시·도·특별자치시·특별자치도 소관(공작물의 설치)(총 8일)\n*접수:특별자치도, 광역시·도(구·시·군), 특별자치시(0일)\n*경유:\n*협의:\n*처리:광역시.도, 특별자치도, 특별자치시(8일)\n8. 특별시 소관(도로의 굴착)(총 10일)\n*접수:서울특별시(구)(0일)\n*경유:\n*협의:\n*처리:서울특별시(구)(10일)\n9. 광역시·도·특별자치시·특별자치도 소관(도로의 굴착)(총 5일)\n*접수:특별자치도, 특별자치시, 광역시·도(구·시·군)(0일)\...",1. 도로법(제61조 제1항) \n2. 도로법 시행규칙(제26조 제2항 별지 제26호),도로점용허가 변경 신청서 도로법 시행규칙 : 별지서식 26호\n,NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 변경내용과 관련된 서류·도면 등
351,1611000-0044,도로점용허가 기간연장,도로점용허가 기간 만료 1개월 전까지 허가기간연장을 신청할 시 제출하는 민원사무입니다.,허가,본인또는대리인신청,기타,"1,000원 또는 행정청이 속하는 지방자치단체의 조례로 정하는 금액",국토교통부,도로관리과,"방문, 우편, FAX, 민원우편, 인터넷","1. 유형없음(총 7일)\n*접수:시.군.구, 지방국토관리청, 특별시·광역시·특별자치도·도, 국토관리사무소, 특별자치시(0일)\n*경유:\n*협의:\n*처리:시.군.구, 지방국토관리청, 특별시·광역시·특별자치도·도, 국토관리사무소, 특별자치시(7일)",1. 도로법(제61조 제1항) \n2. 도로법 시행규칙(제26조 제2항 별지 제25호),도로점용허가 기간연장 신청서 도로법 시행규칙 : 별지서식 25호\n,NaN,NaN
469,1550000-0083,토지·건물 등의 매수청구(고도 보존 및 육성),"역사문화환경 특별보존지구 및 보존육성지구 내 행위 허가를 받지 못하여 본래의 용도로 이용할 수 없게된 자가 사업시행자에게 토지, 건물 등의 매수를 청구하는 민원입니다.",청구,본인또는대리인신청,수수료 없음,수수료 없음,국가유산청,역사문화권과,"방문, 우편, FAX","1. 유형없음(총 40일)\n*접수:특별자치시장, 특별자치도지사, 시장·군수·구청장(1일)\n*경유:\n*협의:문화재청(29일)\n*처리:특별자치시장, 특별자치도지사, 시장·군수·구청장(10일)",1. 고도 보존 및 육성에 관한 특별법(제19조) \n2. 고도 보존 및 육성에 관한 특별법 시행규칙(제8조 및 별지6),토지·건물 등 매수청구서 고도 보존 및 육성에 관한 특별법 시행규칙 : 별지서식 6호\n,NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 토지, 건물 등의 매수사유 발생을 증명하는 서류 1부\n(2) 매수청구인으로서의 자격 구비를 증명하는 서류 1부\n< 공무원확인사항 >\n(3) 대상 토지, 건물 등에 대한 등기사항증명서 1부."
671,1543000-0117,농지의 타용도일시사용 신고(변경신고),타용도로 사용하려는 농지의 소유권을 입증을 위한 신고 사무,신고,본인또는대리인신청,수수료 없음,수수료 없음,농림축산식품부,농지과,"방문, 우편","1. 유형없음(총 10일)\n*접수:특별자치시장, 특별자치도지사, 시장·군수·구청장(0일)\n*경유:\n*협의:\n*처리:특별자치시장, 특별자치도지사, 시장·군수·구청장(10일)",1. 농지법 시행규칙(제27조의2 서식),농지의 타용도일시사용 신고서(변경신고서) 농지법 시행규칙 : 별지서식 27호의 2\n,서식의 '농지의 타용도일시사용신고증' 1부 : 농지법 시행규칙(별지서식 27 호),"1. 농지의 타용도일시사용 신고(변경신고)\n< 민원인 제출서류 >\n(1) 타용도로 사용하려는 기간 등이 표시된 사업계획서\n(2) 타용도로 사용하려는 농지의 소유권을 입증하는 서류(토지 등기 사항증명서로 확인할 수 없는 경우만 해당합니다) 또는 사용권을 가지고 있음을 입증하는 서류(사용승낙서만 해당됩니다)\n(3) 해당 농지의 타용도사용이 농지개량시설 또는 도로의 폐지 및 변경이나 토사의 유출, 폐수의 배출, 악취의 발생 등을 수반하여 인근 농지의 농업경영과 농어촌생활환경의 유지에 피해가 예상되는 경우에는 대체시설의 설치 등 피해방지계획서\n(4) 영 제40조제1항에 따른 복구계획 및 복구비용명세서(변경허가신청의 경우에는 이미 제출한 복구계획과 복구비용명세서의 변경이 필요한 경우만 해당합니다)\n(5) 변경내용을 증명할 수 있는 서류를 포함한 변경사유서(변경허가신청의 경우만 해당합니다)\n< 공무원확인사항 >\n(6) 해당 농지의 토지 등기사항증명서(신고인이 다른 용도로 사용하려는 농지의 소유자인 경우만 해당)"


In [10]:
#담당부서 unique 몇개인지 찾는식
unique_departments = df['담당부서'].unique()
unique_department1s = df['소관부처'].unique()
print(f"Unique departments: {len(unique_departments)}")
print(f"Unique departments: {len(unique_department1s)}")

Unique departments: 753
Unique departments: 46


In [4]:
# 청크 생성
documents = []
metadatas = []
for idx, row in df.iterrows():
    chunk = "\n".join([f"[{col}] {row[col]}" for col in df.columns])
    documents.append(chunk)
    metadatas.append({"사무명": row["사무명"], "분류번호": row["분류번호"]})


In [5]:
# # 임베딩 생성
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# roberta 모델을 사용하여 문서 임베딩 생성 -> 출력 차원: 768

#각 청





In [21]:
embeddings = model.encode(documents, show_progress_bar=True)
# # FAISS 저장
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# # 인덱스와 메타데이터 저장
faiss.write_index(index, "rag_index.faiss")
with open("rag_metadata.pkl", "wb") as f:
    pickle.dump((documents, metadatas), f)

Batches:   0%|          | 0/189 [00:00<?, ?it/s]

In [22]:
# 인덱스 및 메타데이터 로드
index = faiss.read_index("rag_index.faiss")
with open("rag_metadata.pkl", "rb") as f:
    documents, metadatas = pickle.load(f)

In [41]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# 검색 함수
def search_similar_documents(query, top_k=3):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, top_k)
    return [(documents[i], metadatas[i]) for i in I[0]]

# 프롬프트 생성
def build_prompt(query, retrieved_docs):
    context = "\n\n".join([doc for doc, meta in retrieved_docs])
    return f"""다음은 민원 관련 문서입니다:\n\n{context}\n\n질문: {query}\n\n답변:"""

# 사용자 입력 → GPT 응답 생성
def answer_query(query):
    retrieved_docs = search_similar_documents(query)
    prompt = build_prompt(query, retrieved_docs)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 민원 정보를 잘 알려주는 공공서비스 상담 챗봇입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

In [43]:
# 예시 사용
query = "건축물 대장을 발급받으려면 어떻게 해야 하나요?"
response = answer_query(query)
print(response)

건축물 대장을 발급받기 위해서는 다음과 같은 절차를 따르시면 됩니다.

1. 신청 유형: 교부
2. 신청자 자격: 누구나 신청 가능
3. 수수료: 발급(1건당):500원, 열람(1건당):300원, 인터넷 발급(열람)시 무료
4. 접수 방법: 방문, 우편, FAX, 인터넷, 모바일, 무인발급기
5. 접수/처리: 시군구 및 읍면동 출장소에서 접수하며, 처리는 즉시 이루어집니다.
6. 신청서: 건축물대장등.초본발급 및 열람신청서를 작성하셔야 합니다.

발급물 내용으로는 일반건축물대장, 집합건축물대장(표제부), 집합건축물대장(전유부), 건축물대장 총괄 등이 있습니다. 

참고로, 건축물 대장은 건축물의 위치, 구조, 용도, 면적 등 건축물에 관한 중요한 사항을 기록한 공식 문서입니다.


In [ ]:
# GPT 응답 생성
def generate_answer(query):
    retrieved = search_similar_documents(query)
    context = "\n\n".join([doc for doc, meta in retrieved])
    prompt = f"""다음은 민원 관련 문서입니다:\n\n{context}\n\n질문: {query}\n\n답변:"""

    response = openai.ChatCompletion.create(
        model="gpt-4",  # 또는 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "당신은 민원 정보를 잘 알려주는 공공기관 상담 챗봇입니다. "},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']


In [47]:
import streamlit as st
# Streamlit UI
st.title("📄 민원 상담 GPT 챗봇")
query = st.text_input("궁금한 민원 내용을 입력해주세요:")

if st.button("답변 받기") and query:
    with st.spinner("GPT가 답변을 생성 중입니다..."):
        answer = generate_answer(query)
        st.success(answer)

2025-04-30 23:25:01.712 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.752 
  command:

    streamlit run /Users/type/textmining-pj/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-30 23:25:01.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Sessi

In [ ]:
def generate_user_questions_prompt(task_name: str, task_summary: str) -> list:
    return [
        {
            "role": "system",
            "content": (
                "당신은 실제로 어떤 민원을 넣으려는 일반 시민입니다. 아래의 민원 제목과 개요를 참고해서, "
                "현실적인 배경이 있는 질문을 5개 작성하세요. 각 질문은 실제 민원인의 처지에서 상황을 간단히 설명하고, "
                "그 상황에 맞는 질문을 하는 식이어야 합니다.\n\n"
                "예시:\n"
                "- 몇 년 전에 친양자로 입양한 자녀가 있는데 가족 간 갈등으로 인해 입양을 취소하고 싶습니다. 어떤 절차를 따라야 하나요?\n"
                "- 예전에 압류된 차량을 다시 등록하려고 하는데 구비서류가 복잡한지 궁금합니다.\n"
                "- 부모님이 돌아가시면서 상속 때문에 토지이용계획확인서를 떼야 하는데 어디서 신청하나요?\n\n"
                "이런 형식으로 사용자 상황이 담긴 질문 5개를 만들어 주세요."
            )
        },
        {
            "role": "user",
            "content": f"[사무명]: {task_name}\n[사무개요]: {task_summary}"
        }
    ]


def get_generated_questions(task_name, task_summary):
    messages = generate_user_questions_prompt(task_name, task_summary)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=0.7
    )
    raw_output = response.choices[0].message.content
    return [q.strip("-• ").strip() for q in raw_output.strip().split("\n") if q.strip()]

In [ ]:
#RAG용 프롬프트 및 응답 생성
def build_prompt_rag(question: str, docchunk: str) -> str:
    return (
        f"[행정안전부민원안내정보_20250327.xlsx]에서 제공된 민원 문서 내용입니다. 이 문서는 행정 민원 처리 절차를 담고 있습니다.\n\n"
        f"{doc_chunk}\n\n"
        f"위 문서의 정보를 바탕으로 아래 질문에 답해주세요:\n"
        f"{question}\n\n"
        f"주의: 반드시 문서에서 제공된 정보를 최대한 활용하세요. 문서에 해당 민원 내용이 있는 경우, '없다'고 하지 마세요.\n"
        f"민원인이 궁금해하는 신청 자격, 필요 서류, 접수 절차, 수수료, 법적 근거 등 문서의 정보를 친절하게 안내해 주세요.\n\n"
        f"[답변]:"
    )

def answer_query_with_rag(question: str):
    retrieved_docs = search_similar_documents(question, top_k=1)
    doc_text, meta = retrieved_docs[0]
    prompt = build_prompt_rag(question, doc_text)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 민원 정보를 잘 알려주는 공공서비스 상담 챗봇입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response.choices[0].message.content

In [ ]:
#예시: 메타데이터에서 사무명/사무개요를 기반으로
num = [2, 42, 2997]
for i in num:
    meta = metadatas[i]
    if isinstance(meta.get("사무명"), str) and isinstance(documents[i], str):
        task_name = meta["사무명"]
        task_summary_line = [line for line in documents[i].split("\n") if "[사무개요]" in line]
        if task_summary_line:
            task_summary = task_summary_line[0].replace("[사무개요]", "").strip()
            # 질문 생성
            questions = get_generated_questions(task_name, task_summary)

            for q in questions:
                print(f"\n🟨 질문: {q}")
                answer = answer_query_with_rag(q)
                print(f"🟩 답변: {answer}")

In [3]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # 또는 직접 API 키 입력

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "항상 한국어로 짧고 간결하게 대답해 주세요."},
        {"role": "user", "content": "식약처에 내가 만든 음식이 안전하다고 보여주고 싶은데, 전화 어디로 걸어야 해? 전화번호"}
    ]
)

print(response.choices[0].message.content)

식약처의 본부 대표전화는 043-719-2114입니다. 해당 번호로 문의하시면 적절한 부서로 연결해 줄 것입니다.
